---
# Download Echo GPV
---

In [3]:
from multiprocessing import Process
import subprocess
import datetime
from pathlib import Path
import xarray as xr

from src.Ggis1km_downloder import Downloader
from src.util import fix_ds

In [4]:
def convert2nc(bin_file, wgrib2='/home/yoshimi/grib2/wgrib2/wgrib2'):
    nc_file = bin_file.split('.')[0] + ".nc"
    subprocess.run([wgrib2, bin_file, '-netcdf', nc_file])
    
    ds = xr.load_dataset(nc_file)
    ds = fix_ds(ds)
    comp = dict(zlib=True, complevel=9)
    encoding = {var: comp for var in ds.data_vars}
    ds.to_netcdf(nc_file, encoding=encoding)
    
    subprocess.run(['rm', bin_file]) 

def download_gpv(start, end, num='10', timescale='minutes', tar_path='./tar', bin_path='/media/yoshimi/9E9401BB94019745/pixel/radar'):
    dt = datetime.datetime.strptime(start, '%Y/%m/%d %H:%M:%S')
    dt_limit = datetime.datetime.strptime(end, '%Y/%m/%d %H:%M:%S')
    
    downloader = Downloader(tar_path, bin_path)
    while dt < dt_limit:
        downloader.set_date(dt)
        if downloader.get_bin_file():
            convert2nc(downloader.bin_file_path)
        dt = dt + eval('datetime.timedelta(' + timescale + '=' + num + ')')

In [5]:
dates = ['2010/01/01 00:00:00', 
         '2011/01/01 00:00:00', 
         '2012/01/01 00:00:00', 
         '2013/01/01 00:00:00', 
         '2014/01/01 00:00:00', 
         '2015/01/01 00:00:00', 
         '2016/01/01 00:00:00', 
         '2017/01/01 00:00:00', 
         '2018/01/01 00:00:00', 
         '2019/01/01 00:00:00', 
         '2020/01/01 00:00:00', 
         '2021/01/01 00:00:00', ]
process_num = len(dates) - 1 

In [ ]:
%%capture
process_list = []
for i in range(process_num):
    process = Process(
        target=download_gpv,
        kwargs={'start':dates[i],
                'end':dates[i+1]})
    
    process.start()
    process_list.append(process)

for process in process_list:
    process.join()